In [1]:
using Pkg
using CSV
using DataFrames
using Flux
using Flux: onehot, onehotbatch, logitcrossentropy, reset!, throttle
using Statistics: mean
using Random
Pkg.add("Parameters")
using Parameters: @with_kw

   Updating registry at `~/.julia/registries/General`
   Updating 

git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [4]:
@with_kw mutable struct Args
    lr::Float64 = 1e-3     # Learning rate 
    N::Int = 3             # Number of perceptrons in hidden layer
    test_len::Int = 100    # Number of unique words in test data 
    word_list_len::Int = 0 # Total number of unique words
    throttle::Int = 10     # throttle timeout
end

Args

In [5]:
#helper function to parse through each row
#s is a string, d is the dictionary
include("punctuation_strip.jl")
function rowParser(s,d)
    #for every word, using key " ", add to dictionary
    a = split(s, " ")
    for word in a
        modWord = lowercase(word)
        modWord = string_cleaner(modWord)
        if haskey(d, modWord)
            d[modWord] += 1
        else
            d[modWord] = 1
        end
    end
    return nothing
end     

["something with a long set of strings of question at XTIMEX XTIMEX XTIMEX XTIMEX on XDATEX XDATEX 9 XDATEX 99 XDATEX XDATEX 9 ", "some times there are to long strings to to ", "want to remove 2 to 5 word word 2 to 6 2 to 6 10 to 41 19 to 3 to 3 to 3 to 3 to 3", "v 345 vv 3.6 7 v 8 w w 9 1 to 5 1 6 2 3 3 to 6 ounces of enfamil"]


rowParser (generic function with 1 method)

In [6]:
#Add a word to the dictionary if not yet encountered
#Merge all of the dictionaries
function getWords(array, d)
    for row in array #put all of the words into the dictionary
        rowParser(row, d)
    end
    return d
end

getWords (generic function with 1 method)

In [8]:
function process_data()
    df = CSV.File("/Users/rachelwu/Desktop/very_fake_diet_data.csv") |> DataFrame! 
    col1 = df[:, 1] 
    col2 = df[:, 2]
    args = Args()
    
    dict = Dict{String, Integer}()
    #fill dictionary with words from df
    getWords(col1, dict)
    
    #add a string to our dictionary that will be placeholder for words found in testing never seen before
    dict["<unk>"] = 0 
    
    classes = unique(col2)
    args.class_len = length(classes)
    words = collect(skipmissing(keys(dict)))
    args.word_list_len = length(words)

    unique(filter(x -> x ∉ words, join(vcat(values(dict)...))))

    #dataset = [(onehotbatch(w, words, '<unk>'), onehot(l, class_len)) (here split into sentences as outside for loop) for w in words[col1]] |> shuffle

    items = []
    for s in col1
        push!(items, split(s, " "))
    end
    println(items[1])
    #for loops: go through each sentence (which is broken down into individual words), go through classification
    dataset = [(onehotbatch(s, words, "<unk>"), onehot(c, classes)) 
                for (s, c) in zip(items, col2)] |> shuffle
    
    train, test = dataset[1:end-args.test_len], dataset[end-args.test_len+1:end]
    return train, test, args
end

process_data (generic function with 1 method)

In [ ]:
a = Args()
t1 , t2 = process_data(a)
t1